# Geração de Relatórios do Vicarius VRX

## Preparação do Ambiente

### Importando Bibliotecas

In [15]:
%pip install openpyxl
import os, requests, json
from datetime import datetime, timedelta
import time
import pandas as pd
from openpyxl.styles import Alignment, Font


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\CarlosHenriqueFerrei\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Inicializando a conexão

In [16]:
def load_configuration():
    api_key = os.getenv('API_KEY_ESPM')
    api_url = os.getenv('API_URL_ESPM')
    return api_key, api_url

In [17]:
API_KEY, API_URL = load_configuration()
HEADERS = {
    'Vicarius-Token': API_KEY,
    "Content-Type": "application/json"
}
print(API_URL)

https://espm.vicarius.cloud


In [18]:
last_query_time = 0

def control_rate(query_limit=50):
    global last_query_time

    elapsed_time = time.time() - last_query_time
    if elapsed_time < (60 / query_limit):
        time.sleep((60 / query_limit) - elapsed_time)

    last_query_time = time.time()

In [19]:
def getEndpoints(apikey,urldashboard,fr0m,siz3, endpointsMap):

    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }

    params = {
        'from': fr0m,
        'size': siz3,
    }

    try:
        #print(urldashboard)
        response = requests.get(urldashboard + '/vicarius-external-data-api/endpoint/search', params=params, headers=headers)
        parsed = json.loads(response.text) 
        
    except:
        print("something is wrong, will try again....")
    
    strEndpoints = ''
    for i in parsed['serverResponseObject']:
        #print(json.dumps(i,indent=2))
        #print(i['endpointOperatingSystem']['operatingSystemName'])
    
        endpointUpdatedAt = str((i['endpointUpdatedAt']))
        operatingSystemName = (i['endpointOperatingSystem']['operatingSystemName'])
        agentVersion = (i['endpointVersion']['versionName'])
        
        endpointsMap[i['endpointHash']] = [('Hostname', i['endpointName']), ('Sistema Operacional', operatingSystemName), ('Agent Version', agentVersion)]
        
        strEndpoints += (str(i['endpointId']) + "," + i['endpointName'] + "," + i['endpointHash'] + "," + operatingSystemName + "," + agentVersion + "," + endpointUpdatedAt +"\n")
    
    return strEndpoints



In [20]:
endpointsMap = {}
control_rate(20)
strEndpoints = getEndpoints(API_KEY, API_URL, 0, 499, endpointsMap)
print(strEndpoints)
for endpoint in endpointsMap:
    print(endpointsMap[endpoint])

1001838,SPAA103,475db8680e385214e0b73e278c7e9fb9,Windows Server 2022 Datacenter,5.1.20,1733511283590
1009114,AATISEGINF-02,2332438c1daadc5ef42f087d2f62a6e1,Microsoft Windows 11 Pro,5.1.20,1734030849241
586507,RJRO092,aee154a395e6c637c18f9f8632b98cf6,Windows Server 2016,5.1.18,1732811592605
876719,AATISEGINFNT-09,d359adf1210255299d7ba1485204e02a,Windows 11,5.1.20,1733445633651
652699,SPAA103,48aa0b1cff0d85a7048b505378f75d32,Windows Server 2022,5.1.20,1734524561366
895910,SPAA204,01182cce5361d542f308828f888efc5f,Windows Server 2016,5.1.18,1734641476869
630823,AATISEGINFNT-02,5a8ab1eaf75ced4f0824b9dbc907faff,Windows 11,5.1.20,1733424977303
593774,SPAA171,00d97cd11eb9b7b22dab44dfe25e4b43,Windows Server 2012,5.1.20,1737470064375
681268,SPAA93,9c3bfe2f60de750a0bbc53b77f0c2abb,Windows Server 2016,5.1.20,1738160869891
869190,SPAA129,795f1a9b7d9304435ca8fae3d68fe4e8,Windows Server 2016,5.1.24,1739284060017
637567,SPVID002,821e93b75c31e2617282f83e3dbd79fd,Windows 10,5.1.24,1739242161020
214447,S

In [21]:
def getEndpointAttributes(apikey, urldashboard, fr0m, siz3, endpointHash):

    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }

    params = {
        'from': fr0m,
        'size': siz3,
        'q': 'endpointAttributesEndpoint.endpointHash==' + endpointHash
    }

    try:
        response = requests.get(urldashboard + '/vicarius-external-data-api/endpointAttributes/search', params=params, headers=headers)
        parsed = json.loads(response.text) 
        
    except:
        print("something is wrong, will try again....")
    
    print(json.dumps(parsed,indent=4))
    return parsed


In [ ]:
for key in endpointsMap:
    control_rate()
    jresponse = getEndpointAttributes(API_KEY, API_URL, 0, 500, '25c6646c418c7cb0b28501377fde97d4')

    ipAddresses = ''

    for obj in jresponse['serverResponseObject']:
        for line in obj:
            if line == 'endpointAttributesAttribute':
                if 'IP' in obj[line]['attributeAttributeSource']['attributeSourceName']:
                    ipAddresses += obj[line]['attributeExternalId'] + ', '
                    
    endpointsMap[key].append(('IP', ipAddresses[:-2]))
    

{
    "serverResponseResult": {
        "serverResponseResultCode": "SUCCESS",
        "serverResponseResultExecutionTime": 469
    },
    "serverResponseCount": 0,
    "serverResponseObject": []
}
{
    "serverResponseResult": {
        "serverResponseResultCode": "SUCCESS",
        "serverResponseResultExecutionTime": 6
    },
    "serverResponseCount": 0,
    "serverResponseObject": []
}
{
    "serverResponseResult": {
        "serverResponseResultCode": "SUCCESS",
        "serverResponseResultExecutionTime": 517
    },
    "serverResponseCount": 0,
    "serverResponseObject": []
}
{
    "serverResponseResult": {
        "serverResponseResultCode": "SUCCESS",
        "serverResponseResultExecutionTime": 7
    },
    "serverResponseCount": 0,
    "serverResponseObject": []
}
{
    "serverResponseResult": {
        "serverResponseResultCode": "SUCCESS",
        "serverResponseResultExecutionTime": 4
    },
    "serverResponseCount": 0,
    "serverResponseObject": []
}
{
    "serverRes

KeyError: 'serverResponseObject'

In [ ]:
def getCountEndpoints(apikey,urldashboard):

    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }

    params = {
        'from': 0,
        'size': 1,
        'sort': '+endpointId'
    }

    try:
        response = requests.get(urldashboard + '/vicarius-external-data-api/endpoint/search', params=params, headers=headers)
        jsonresponse = json.loads(response.text)
        responsecount = jsonresponse['serverResponseCount']
        firstID = jsonresponse['serverResponseObject'][0]['endpointId']

    except:
        print("something is wrong, will try again....")
        print("response: ",response.text)

    return responsecount,firstID


In [ ]:
def getEndpointsPatches(apikey,urldashboard,fr0m,siz3,endpointHash):

    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }

    params = {
        'from': fr0m,
        'size': siz3,
        'group': 'organizationEndpointExternalReferenceExternalReferencesPatches.patchName.raw;organizationEndpointExternalReferenceExternalReferencesPatches.patchDescription;organizationEndpointExternalReferenceExternalReferencesPatches.patchSensitivityLevel.sensitivityLevelName;organizationEndpointExternalReferenceExternalReferencesPatches.patchSensitivityLevel.sensitivityLevelRank;externalReferenceId;>;organizationEndpointExternalReferenceExternalReferencesPatches.patchId;externalReferenceSourceId;endpointId',
        'includeOriginalDoc': 'false',
        'newParser': 'true',
        'objectName': 'OrganizationEndpointExternalReferenceExternalReferences',
        'sort': 'OrganizationEndpointExternalReferenceExternalReferences.sensitivityLevelRank',
        'subAggregationLevel': '0',
        'sumLastSubAggregationBuckets': '0',
        'q': 'organizationEndpointExternalReferenceExternalReferencesEndpoint.endpointHash=in=('+endpointHash+')',
    }

    try:
        response = requests.get(urldashboard + '/vicarius-external-data-api/aggregation/searchGroup?', params=params, headers=headers)
        parsed = json.loads(response.text)
          
    except:
        print("something is wrong, will try again....")
        return endpointHash

    str_endpoint_patches = ""
    str_endpoint_app_patches = ""
    str_endpoint_so_patches = ""
    i = j = k = 0
    for obj in parsed['serverResponseObject']:
        patchDescriptions = ''
        
        for agg in obj['aggregationAggregations']:
            if 'patchDescriptions' in agg['aggregationName']:
                patchDescriptions = agg['aggregationId']
                
        if any(keyword in patchDescriptions.lower() for keyword in ["kb"]):
            j += 1
            str_endpoint_so_patches += f"[{j}] " + patchDescriptions + "; \n"
        else:
            k += 1
            str_endpoint_app_patches += f"[{k}] " + patchDescriptions + "; \n"
        
        i += 1
        str_endpoint_patches += f"[{i}] " + patchDescriptions + "; \n"
    
    totalPatchs = len(parsed['serverResponseObject'])
    return totalPatchs, str_endpoint_patches[:-3], str_endpoint_so_patches[:-3], str_endpoint_app_patches[:-3]


In [ ]:
for key in endpointsMap:
    control_rate(50)
    patches_count, patches, so_patches, app_patches = getEndpointsPatches(API_KEY, API_URL, 0, 500, key)
    endpointsMap[key].append(('Atualizações', patches_count))
    endpointsMap[key].append(('Atualizações do SO', so_patches))
    endpointsMap[key].append(('Atualizações de Apps', app_patches))
    #print(endpointsMap[key])
    

In [ ]:
def getCountEventsPerAsset(apikey, urldashboard, fr0m, siz3, endpointHash, trycount=0):
    errors = []
    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }
    
    params = {
        'from': fr0m,
        'size': siz3,
        'q': 'organizationEndpointVulnerabilitiesEndpoint.endpointHash=in=('+endpointHash+')',
        'includeFields' : 'organizationEndpointVulnerabilitiesEndpoint.endpointId,organizationEndpointVulnerabilitiesEndpoint.endpointHash,organizationEndpointVulnerabilitiesVulnerability.vulnerabilityExternalReference.externalReferenceExternalId,organizationEndpointVulnerabilitiesVulnerability.vulnerabilityId,organizationEndpointVulnerabilitiesProduct.productName,organizationEndpointVulnerabilitiesOperatingSystem.operatingSystemName,organizationEndpointVulnerabilitiesVersion.versionName,organizationEndpointVulnerabilitiesSubVersion.subVersionName,organizationEndpointVulnerabilitiesProductRawEntry.productRawEntryName,organizationEndpointVulnerabilitiesVulnerability.vulnerabilitySensitivityLevel.sensitivityLevelName,organizationEndpointVulnerabilitiesVulnerability.vulnerabilitySummary,organizationEndpointVulnerabilitiesEndpoint.endpointName,organizationEndpointVulnerabilitiesPatch.patchId,organizationEndpointVulnerabilitiesPatch.patchName,organizationEndpointVulnerabilitiesPatch.patchReleaseDate,organizationEndpointVulnerabilitiesCreatedAt,organizationEndpointVulnerabilitiesUpdatedAt,organizationEndpointVulnerabilitiesVulnerability.vulnerabilityV3ExploitabilityLevel,organizationEndpointVulnerabilitiesVulnerability.vulnerabilityV3BaseScore'
    }
    if (trycount < 2):
        try:
            response = requests.get(urldashboard + '/vicarius-external-data-api/organizationEndpointVulnerabilities/search', params=params, headers=headers)
            if response.status_code == 429:
                print("API Rate Limit exceeded ... Waiting and Trying again")
                errors.append("API Rate Limit")
                time.sleep(60)
                trycount += 1
                getCountEventsPerAsset(apikey,urldashboard,endpointHash,trycount)
            jsonresponse = json.loads(response.text)
        except Exception as e:
                print(f'something is wrong, will try again- EndpointHash: {endpointHash}, ')
                errors.append(f"Exception: {e}, EndpointHash: {endpointHash}")
                time.sleep(60)
                trycount += 1
                getCountEventsPerAsset(apikey,urldashboard,endpointHash,trycount)
    #print (jsonresponse)
    
    try:
        responsecount = int(jsonresponse['serverResponseCount'])
    except:
        responsecount = 0
        
    
    try: 
        return responsecount, jsonresponse, errors
    except Exception as e:
        errors.append(f"Return Exception: {e},")
        jsonresponse = {}
        return 0, jsonresponse, errors

In [ ]:
for key in endpointsMap:
    control_rate()
    eventsCount, response, errors = getCountEventsPerAsset(API_KEY, API_URL, 0, 500, key)
    endpointsMap[key].append(('CVE\'s', eventsCount))
    print(endpointsMap[key])
    

## Relatório de Vulnerabilidades

In [ ]:
def convert_timestamp_to_datetime(timestamp):
    try:
        # Check if the timestamp is likely in milliseconds (large numbers)
        if timestamp > 1e10:  # Adjust threshold as necessary
            timestamp /= 1000.0  # Convert from milliseconds to seconds

        return datetime.fromtimestamp(timestamp)
    except (TypeError, ValueError, OverflowError):
        return timestamp

In [ ]:
def getEndpointVulnerabilities(apikey,urldashboard,fr0m,siz3,minDate,maxDate,endpointName,endpointHash):

    headers = {
        'Accept': 'application/json',
        'Vicarius-Token': apikey,
    }

    
    
    params = {
        'from': fr0m,
        'size': siz3,
        'q': f"organizationEndpointVulnerabilitiesCreatedAt>{minDate};"
             f"organizationEndpointVulnerabilitiesCreatedAt<{maxDate};"
             f"organizationEndpointVulnerabilitiesEndpoint.endpointHash=in=({endpointHash})",
        'sort' : '-organizationEndpointVulnerabilitiesCreatedAt',
    }
    
    #print(params)
    jresponse = []
    try:
        time.sleep(1)
        response = requests.get(urldashboard + '/vicarius-external-data-api/organizationEndpointVulnerabilities/search', params=params, headers=headers)
        jresponse = json.loads(response.text)
  
    except:
        print("something is wrong, will try again....")
        time.sleep(30)
        getEndpointVulnerabilities(apikey,urldashboard,fr0m,siz3,minDate,maxDate,endpointName,endpointHash)

    return jresponse

In [ ]:
def parseEndpointVulnerabilities(jresponse):
    strVulnerabilities = ''
    vulnReport = []
    index = 0

    for i in jresponse['serverResponseObject']:
        index +=1
        cve = i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilityExternalReference']['externalReferenceExternalId']
        vulid = str(i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilityId'])
        link = 'https://www.vicarius.io/vsociety/vulnerabilities/'+ vulid + '/' + cve
        # fix for empty product name for OS
        try:
            productName = i['organizationEndpointVulnerabilitiesProduct']['productName']
        except:
            try: 
                productName = i['organizationEndpointVulnerabilitiesOperatingSystem']['operatingSystemName']
            except:
                productName = ""
    
        productRawEntryName = i['organizationEndpointVulnerabilitiesProductRawEntry']['productRawEntryName']
        sensitivityLevelName = i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilitySensitivityLevel']['sensitivityLevelName']
        vulnerabilitySummary = i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilitySummary'] 
        
        asset = i['organizationEndpointVulnerabilitiesEndpoint']['endpointName']

        if i['organizationEndpointVulnerabilitiesPatch']['patchId'] > 0:
            patchid = str(i['organizationEndpointVulnerabilitiesPatch']['patchId'])
            patchName = (i['organizationEndpointVulnerabilitiesPatch']['patchName'])
            #handle exception when patchReleaseDate not present
            try:
                patch_release_date = i['organizationEndpointVulnerabilitiesPatch']['patchReleaseDate']
                patch_release_date = str(convert_timestamp_to_datetime(patch_release_date).strftime("%d-%m-%Y"))
            except: 
                patch_release_date = "n\\a"
        else:
            patchid = "n\\a"
            patchName = "n\\a"
            patch_release_date = "n\\a"

        try:
            create_at = i['organizationEndpointVulnerabilitiesCreatedAt']
            update_at = i['organizationEndpointVulnerabilitiesUpdatedAt']
            create_at = str(convert_timestamp_to_datetime(create_at).strftime("%d-%m-%Y"))
            update_at = str(convert_timestamp_to_datetime(update_at).strftime("%d-%m-%Y"))
        except:
            create_at = ""
            update_at = ""

        productName = productName.replace(',',"").replace(";","")
        productRawEntryName = str(productRawEntryName).replace(',',"").replace(";","")
        vulnerabilitySummary = str(vulnerabilitySummary).replace("\"","'")
        vulnerabilitySummary = vulnerabilitySummary.replace("\r","").replace("\n",">>")
        vulnerabilitySummary = vulnerabilitySummary.replace(",","").replace(";","")
        
        #threatLevelId = str(i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilitySensitivityLevel']['threatLevelId'])
        vulnerabilityV3ExploitabilityLevel = i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilityV3ExploitabilityLevel']
        vulnerabilityV3BaseScore = i['organizationEndpointVulnerabilitiesVulnerability']['vulnerabilityV3BaseScore']

        #for group in endpointGroups.split("|"):
        strVulnerabilities += (f'[{index}] ' + sensitivityLevelName + "," + cve + ","+ str(update_at) + "," + link + ",\"" + vulnerabilitySummary + "\";" + "\n")
        vulnReport.append([('Hostname', asset), ('Product Name', productName), ('CVE Severity', sensitivityLevelName), ('CVE', cve), ('Vulnerability ID', vulid), ('Patch ID', patchid), ('Patch Name', patchName), ('Patch Release Date', patch_release_date), ('Vulnerability Creation Date', create_at), ('Last Updated', update_at), ('Link', link), ('Description', vulnerabilitySummary), ('V3 Base Score', vulnerabilityV3BaseScore), ('Exploitability Level', vulnerabilityV3ExploitabilityLevel)])

        maxDate = create_at

    return ('Vulnerabilities', strVulnerabilities), vulnReport


In [ ]:
vulnReport = []

for key in endpointsMap:
    dateNow = str(int(float(datetime.now().timestamp() * 1000)))
    thirty_days_ago = str(int(float((datetime.now() - timedelta(days=15)).timestamp() * 1000)))
    
    control_rate()
    jresonse = getEndpointVulnerabilities(API_KEY, API_URL, 0, 500, thirty_days_ago, dateNow, endpointsMap[key][1][1], key)
    
    assetVulnerabilities, assetVulnerabilitiesReport = parseEndpointVulnerabilities(jresonse)
    #endpointsMap[key].append(assetVulnerabilities)
    
    if len(assetVulnerabilitiesReport) > 0:
        vulnReport.extend(assetVulnerabilitiesReport)
    
    print(assetVulnerabilities[1])
    print(assetVulnerabilitiesReport, '\n')

In [ ]:
for key in endpointsMap:
    print(endpointsMap[key])

In [ ]:
for asset in vulnReport:
    for vuln in asset:
        print(vuln)

In [ ]:
assets_data = [{key: value for key, value in asset} for asset in endpointsMap.values()]
df_hosts = pd.DataFrame(assets_data)
print(df_hosts.columns)
df_hosts

In [ ]:
vuln_data = [{key: value for key, value in vuln} for vuln in vulnReport]
df_vuln = pd.DataFrame(vuln_data)
print(df_vuln.columns)
df_vuln

### Formatar tabelas e exportar para excel

In [ ]:
column_widths = {
    "Hosts": {
        'Hostname': 17, 
        'Sistema Operacional': 15, 'IP': 16, 
        'Agent Version': 11, 'CVE\'s': 11, 'Atualizações': 12, 
        'Atualizações do SO': 50, 'Atualizações de Apps': 50
    },
    "Vulnerabilities": {
        'Hostname': 17, 
        'Product Name': 10, 'Patch Name': 22, 
        'Patch ID': 10, 'CVE': 15, 'CVE Severity': 10, 'V3 Base Score': 10, 'Exploitability Level': 12, 
        'Vulnerability ID': 12, 'Patch Release Date': 12, 'Vulnerability Creation Date': 15, 
        'Last Updated': 12,  
        'Link': 40, 
        'Description': 50
    }
}

In [ ]:
with pd.ExcelWriter("vrx_reports.xlsx", engine="openpyxl") as writer:
    df_hosts.to_excel(writer, sheet_name="Hosts", index=False)
    df_vuln.to_excel(writer, sheet_name="Vulnerabilities", index=False)
    
    workbook = writer.book

    worksheet_hosts = writer.sheets["Hosts"]
    for col_num, col_name in enumerate(df_hosts.columns, start=1):
        col_letter = chr(64 + col_num)  
        width = column_widths["Hosts"].get(col_name, 15)
        worksheet_hosts.column_dimensions[col_letter].width = width
        
        cell = worksheet_hosts[f"{col_letter}1"]
        cell.alignment = Alignment(horizontal="center", vertical="center", wrapText=True)
        cell.font = Font(bold=True)
        
        for row in worksheet_hosts.iter_rows(min_row=1, max_row=worksheet_hosts.max_row, min_col=col_num, max_col=col_num):
            for cell in row:
                cell.alignment = Alignment(wrapText=True)

    worksheet_vuln = writer.sheets["Vulnerabilities"]
    for col_num, col_name in enumerate(df_vuln.columns, start=1):
        col_letter = chr(64 + col_num)
        width = column_widths["Vulnerabilities"].get(col_name, 15)
        worksheet_vuln.column_dimensions[col_letter].width = width

        cell = worksheet_vuln[f"{col_letter}1"]
        cell.alignment = Alignment(horizontal="center", vertical="center", wrapText=True)
        cell.font = Font(bold=True)

        for row in worksheet_vuln.iter_rows(min_row=2, max_row=worksheet_vuln.max_row, min_col=col_num, max_col=col_num):
            for cell in row:
                cell.alignment = Alignment(wrapText=True)

print("Arquivo salvo como vrx_reports.xlsx")